In [41]:
def preprocess_data(data):
    # Drop columns with NaN values
    data = data.dropna(axis=1)

    categorical_columns = ["Materialnummer", "Lieferant OB", "Vertragsposition OB", "Beschaffungsart", "Disponent", "Einkäufer", "Dispolosgröße", "Werk OB", "Warengruppe", "Basiseinheit"]
    numerical_columns = ["Planlieferzeit Vertrag", "Vertrag Fix1", "Vertrag_Fix2", "Gesamtbestand", "Gesamtwert", "Preiseinheit", "WE-Bearbeitungszeit", "Planlieferzeit Mat-Stamm"]

    data[categorical_columns] = data[categorical_columns].astype('category')
    data[numerical_columns] = data[numerical_columns].astype('int64')

    # Select only numeric columns
    processed_data = data[numerical_columns]

    # Normalize features
    #scaler = StandardScaler()
    #processed_data = pd.DataFrame(scaler.fit_transform(processed_data), columns=processed_data.columns)

    return processed_data  

file_path = '../Stammdaten.csv'
df = pd.read_csv(file_path, low_memory=False)
processed_df = preprocess_data(df)

C:\Users\raoul\AppData\Local\Temp\ipykernel_37108\1161721549.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[categorical_columns] = data[categorical_columns].astype('category')
C:\Users\raoul\AppData\Local\Temp\ipykernel_37108\1161721549.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[numerical_columns] = data[numerical_columns].astype('int64')


In [42]:
import numpy as np
from sklearn.cluster import KMeans

def detect_anomalies(data, num_clusters, threshold):
    # Fit KMeans clustering model
    kmeans = KMeans(n_clusters=num_clusters)
    kmeans.fit(data)
    
    # Calculate distances of each point to its nearest cluster center
    distances = kmeans.transform(data)
    
    # Calculate average distance to cluster centers for each point
    avg_distances = np.mean(distances, axis=1)
    
    # Calculate threshold for anomaly detection
    anomaly_threshold = np.percentile(avg_distances, threshold)
    
    # Identify anomalies
    anomalies = data[avg_distances > anomaly_threshold]
    
    return anomalies

anomalies = detect_anomalies(processed_df, num_clusters=5, threshold=95)
print(anomalies)

C:\Users\raoul\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


        Planlieferzeit Vertrag  Vertrag Fix1  Vertrag_Fix2  Gesamtbestand  \
32                           0             0             0              0   
2308                         0             0             0              0   
2309                         0             0             0              0   
2310                         0             0             0              0   
2311                         0             0             0              0   
...                        ...           ...           ...            ...   
184176                       4             4            60        4552510   
185420                     150           150           150          16020   
185490                     112             0             0              0   
185556                      42             0             0              0   
186438                      14            14            84          44060   

        Gesamtwert  Preiseinheit  WE-Bearbeitungszeit  \
32               0

In [43]:
anomalies.to_csv('anomalies.csv', index=False)

In [20]:
processed_df

,Planlieferzeit Vertrag,Vertrag Fix1,Vertrag_Fix2,Gesamtbestand,Gesamtwert,Preiseinheit,WE-Bearbeitungszeit,Planlieferzeit Mat-Stamm
0,0,0,0,0,0,1,1,3
1,0,0,0,0,0,1,1,3
2,0,0,0,0,0,1,1,84
3,0,0,0,0,0,1,1,84
4,0,0,0,0,0,1,1,3
...,...,...,...,...,...,...,...,...
186751,63,0,0,530,362748,100,1,63
186752,4,4,60,67220,739293,100,1,7
186753,4,4,60,2000,29624,100,1,21
186754,4,4,60,17980,7181,100,1,7


In [36]:
anomalies = detect_anomalies(processed_df, num_clusters=5, threshold=95)

C:\Users\raoul\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [38]:
anomalies

,Planlieferzeit Vertrag,Vertrag Fix1,Vertrag_Fix2,Gesamtbestand,Gesamtwert,Preiseinheit,WE-Bearbeitungszeit,Planlieferzeit Mat-Stamm
32,0,0,0,0,0,1000,1,3
2308,0,0,0,0,0,0,1,999
2309,0,0,0,0,0,0,1,999
2310,0,0,0,0,0,0,1,999
2311,0,0,0,0,0,0,1,999
...,...,...,...,...,...,...,...,...
184176,4,4,60,4552510,111006,1000,1,4
185420,150,150,150,16020,115344675,1,1,84
185490,112,0,0,0,0,1,1,999
185556,42,0,0,0,0,1,1,999


In [39]:
processed_df

,Planlieferzeit Vertrag,Vertrag Fix1,Vertrag_Fix2,Gesamtbestand,Gesamtwert,Preiseinheit,WE-Bearbeitungszeit,Planlieferzeit Mat-Stamm
0,0,0,0,0,0,1,1,3
1,0,0,0,0,0,1,1,3
2,0,0,0,0,0,1,1,84
3,0,0,0,0,0,1,1,84
4,0,0,0,0,0,1,1,3
...,...,...,...,...,...,...,...,...
186751,63,0,0,530,362748,100,1,63
186752,4,4,60,67220,739293,100,1,7
186753,4,4,60,2000,29624,100,1,21
186754,4,4,60,17980,7181,100,1,7
